In [ ]:
from sentinel_2 import data_manager, Indices
from crops_manager import crops_manager
from plot_utils import *

from shapely.geometry import Point, Polygon
import matplotlib.pyplot as plt
from datetime import datetime
from skimage import exposure
import rasterio.mask
import pandas as pd
import numpy as np
import matplotlib
import rasterio
import re
import os

In [ ]:
# read VH data
data = pd.read_csv('./dati.csv', sep=';')

data.loc[:, 'lat'] = data['COORD. GPS'].apply(lambda x: float(x.split(', ')[0]))
data.loc[:, 'long'] = data['COORD. GPS'].apply(lambda x: float(x.split(', ')[1]))

data.loc[:, 'dateref'] = data['DATA EVENTO'].apply(lambda x: [i for i in re.sub('  +', '_', x).split('_') if len(i) > 0])
data = data.explode('dateref')
data['dateref'] = data.dateref.apply(lambda x: datetime.strptime(x, '%d/%m/%Y ; %H:%M'))

In [ ]:
data.head()

In [ ]:
numpy_root_data_dir = '/Volumes/cuda4.deepleey.com/Sentinel2-crops'

tile = 'T32TNQ'

In [ ]:
cmgr = crops_manager(numpy_root_data_dir, tile)

### data una coppia di coordinate recupero l'id del crop che le contiene

In [ ]:
data.loc[:, 'crop'] = data.apply(lambda x: cmgr.getId_by_longlat(x.long, x.lat), axis=1)
data = data.explode('crop').dropna().reset_index(drop=True)

### dato l'id del crop, restituisce gli shape dei campi individuati

In [ ]:
semantic_map = {_id: cmgr.get_veg_shapes(_id, seg_type='semantic') for _id in data.crop.unique()}
panoptic_map = {_id: cmgr.get_veg_shapes(_id, seg_type='panoptic') for _id in data.crop.unique()}

In [ ]:
data.loc[:, 'semantic_label'] = data.apply(lambda x: semantic_map[x.crop][semantic_map[x.crop].contains(Point(x.long, x.lat))].label.values 
                                           if semantic_map[x.crop].shape[0] > 0 else None, axis=1).explode()

In [ ]:
data.loc[:, 'panoptic_label'] = data.apply(lambda x: panoptic_map[x.crop][panoptic_map[x.crop].contains(Point(x.long, x.lat))].label.values 
                                           if panoptic_map[x.crop].shape[0] > 0 else None, axis=1).explode()

In [ ]:
i_event = data.groupby('CERTIFICATO').apply(lambda x: pd.Series(index=x.index, data=list(range(len(x))))).reset_index(level=0, drop=True).rename('i_event')

In [ ]:
data = pd.concat([data, i_event], axis=1)

In [ ]:
def panel_period_close_to_event(crop, dateref, long, lat):
    f = os.path.join(numpy_root_data_dir, tile, crop)
    dm = data_manager(f)
    date_df = dm.__get_df_from_filelist__(dm.data_fnames)
    
    serie_to_plot = pd.concat([date_df[date_df.index < dateref].iloc[-5:], 
                               date_df[date_df.index > dateref].iloc[:5]], axis=0)
    
    geo_ref = cmgr.__retrieve_geo_ref__(crop, count=3)
    row, col = geo_ref.index(long, lat)

    fig, ax = plt.subplots(2, 5, figsize=(20, 8))
    for i, (idx, elm) in enumerate(serie_to_plot.items()):
        x = np.load(elm)
        im = get_rgb(x)
        ax[i//5][i%5].imshow(im, interpolation=None)
        ax[i//5][i%5].plot(col, row, marker='.', c='r')
        ax[i//5][i%5].axis('off')
        ax[i//5][i%5].set_title(idx.date())

In [ ]:
panel_period_close_to_event(**data.iloc[10][['crop', 'dateref', 'long', 'lat']])

In [ ]:
def add_veg_shape(_ax, crop, seg_type='semantic'):
    _veg = cmgr.get_veg_shapes(crop, seg_type=seg_type, coord=False)
    for _, row in _veg.iterrows():
        _interior, _exterior = get_plt_patch(row.label, row.geometry)
        _ax.add_patch(_interior)
        _ax.add_patch(_exterior)

In [ ]:
def plot_legend(_ax):
    _ax.matshow(np.stack([np.arange(0, 20) for _ in range(3)], axis=1), cmap = cmap)
    _ax.set_yticks(ticks = range(20), labels=['{} - {}'.format(i, _l) for i, _l in enumerate(label_names)])
    _ax.set_xticks(ticks=[])

In [ ]:
def panel_crops(crop, long, lat):
    f = os.path.join(numpy_root_data_dir, tile, crop)
    dm = data_manager(f)
    x = dm.get_data_from_date(datetime(2022, 7, 10)).astype(np.float64)
    fig, ax = plt.subplots(1, 4, figsize=(25, 8))
    im = get_rgb(x)
    ax[0].imshow(im)
    ax[1].imshow(im)
    ax[2].imshow(im)
    
    add_veg_shape(ax[1], crop, seg_type='semantic')
    add_veg_shape(ax[2], crop, seg_type='panoptic')
    
    geo_ref = cmgr.__retrieve_geo_ref__(crop, count=3)
    row, col = geo_ref.index(long, lat)
    ax[0].plot(col, row, marker='.', c='r')
    ax[1].plot(col, row, marker='.', c='r')
    ax[2].plot(col, row, marker='.', c='r')

    ax[0].axis('off')
    ax[1].axis('off')
    ax[2].axis('off')
    
    plot_legend(ax[3])

In [ ]:
panel_crops(**data.iloc[10][['crop', 'long', 'lat']])